In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/projeto/'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark DataFrames and SQL") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
hdfs_path = "hdfs://hdfs-nn:9000/projeto/bronze/players/seasons.csv"
hdfs_path_auxiliar = "hdfs://hdfs-nn:9000/projeto/bronze/draft/draft_year.csv"
hdfs_path_pl = "hdfs://hdfs-nn:9000/projeto/bronze/draft/NBA_Full_Draft_1947-2018.csv"
hdfs_path_birthday = "hdfs://hdfs-nn:9000/projeto/bronze/players/birthdays.csv"


In [ ]:
dataFrame = spark.read.csv(hdfs_path, header= True)
dataFrame_auxiliar = spark.read.csv(hdfs_path_auxiliar, header= True)
dataFrame_pl = spark.read.csv(hdfs_path_pl, header= True)
dataFrame_terciario= spark.read.csv(hdfs_path_birthday, header= True)

In [ ]:
#TRATAR O PRINCIPAL DATASET 

In [ ]:
dataFrame.printSchema()

In [ ]:
dataFrame.toPandas()

In [ ]:
dataFrame.createOrReplaceTempView("dataframe")

# Then you can write all the SQL you want, as your json is seen as a regular table now.
sqlized_df = spark.sql(
    """
    SELECT pick
    FROM dataframe
    where reference =="nesmiaa01"
    """
)

sqlized_df.toPandas()

In [ ]:
dataFrame=dataFrame.drop('ws-dum')
dataFrame=dataFrame.drop('bpm-dum')

In [ ]:
dataFrame.printSchema() 

In [ ]:
dataFrame= dataFrame.withColumnRenamed("_c0","reference")\
                    .withColumnRenamed("g","games")\
                    .withColumnRenamed("mp","minutes_played")\
                    .withColumnRenamed("per","player_efficiency")\
                    .withColumnRenamed("ts_pct","true_shooting_percentage")\
.withColumnRenamed("fg3a_per_fga_pct","proportion_of_shots_taken_3pts")\
                    .withColumnRenamed("fta_per_fga_pct","free_throws_attempted_per_field_goal_attempt")\
                    .withColumnRenamed("orb_pct","offensive_rebounds_pct")\
                    .withColumnRenamed("drb_pct","defensive_rebounds_pct")\
.withColumnRenamed("total_rebounds","total_rebounds")\
                    .withColumnRenamed("ast_pct","assists_pct")\
                    .withColumnRenamed("stl_pct","steals_pct")\
                    .withColumnRenamed("blk_pct","blocks_pct")\
.withColumnRenamed("tov_pct","Turnovers")\
                    .withColumnRenamed("usg_pct","usage_pct")\
                    .withColumnRenamed("ows","offensive_win_shares")\
                    .withColumnRenamed("dws","defensive_win_shares")\
.withColumnRenamed("ws","win_shares")\
                    .withColumnRenamed("obpm","offensive_box_plus_minus")\
                    .withColumnRenamed("dbpm","defensive_box_plus_minus")\
                    .withColumnRenamed("Box_plus_minus","Box_plus_minus")\
.withColumnRenamed("Value Over Replacement Player","value_over_replacement_player")

In [ ]:
from pyspark.sql.functions import when, col, concat, lit
#Adicionar coluna para ver os minutos todos e a percentagem de minutos jogados (dos jogos que ele foi convocado)
dataFrame1 = dataFrame.withColumn(    
    "percentage_minutes_game",
    col("minutes_played")/ (col("games")*48) *100
)




In [ ]:
#Percentagem de minutos ao longo da epoca
from pyspark.sql.functions import when, col, concat, lit

dataFrame2 = dataFrame1.withColumn(    
    "percentage_minutes_game_season",
    col("minutes_played")/ (72*48) *100
)



In [ ]:
dataFrame2.toPandas()

In [ ]:
from pyspark.sql.functions import substring, length, col, expr
dataFrame3 = dataFrame2.withColumn("reference",expr("substring(reference, 1, length(reference)-3)"))


In [ ]:
#TRATAR O SECUNDARIO---------------------------------------------------------------------------------

In [ ]:
dataFrame_auxiliar.toPandas()

In [ ]:
dataFrame_auxiliar.printSchema()


In [ ]:
#neste dataset so me interessa o id a pick e o ano da pick portanto é necessario realizar um filtro e os jogos

In [ ]:
dataFrame_auxiliar= dataFrame_auxiliar.withColumnRenamed("_c0","b_reference")\
                    .withColumnRenamed("g","games_total_seasons")\
                    .withColumnRenamed("player","name")\
                    .withColumnRenamed("Year","draft_year")

In [ ]:
dataFrame_auxiliar= dataFrame_auxiliar.drop('team_id')
dataFrame_auxiliar= dataFrame_auxiliar.drop('vorp')
dataFrame_auxiliar= dataFrame_auxiliar.drop('bpm')
dataFrame_auxiliar= dataFrame_auxiliar.drop('ws_per_48')
dataFrame_auxiliar= dataFrame_auxiliar.drop('ws')

dataFrame_auxiliar= dataFrame_auxiliar.drop('ast_per_g')
dataFrame_auxiliar= dataFrame_auxiliar.drop('trb_per_g')
dataFrame_auxiliar= dataFrame_auxiliar.drop('mp_per_g')
dataFrame_auxiliar= dataFrame_auxiliar.drop('pts_per_g')
dataFrame_auxiliar= dataFrame_auxiliar.drop('ft_pct')

dataFrame_auxiliar= dataFrame_auxiliar.drop('fg3_pct')
dataFrame_auxiliar= dataFrame_auxiliar.drop('fg_pct')
dataFrame_auxiliar= dataFrame_auxiliar.drop('ast')
dataFrame_auxiliar= dataFrame_auxiliar.drop('trb')
dataFrame_auxiliar= dataFrame_auxiliar.drop('pts')


dataFrame_auxiliar= dataFrame_auxiliar.drop('mp')
 #De recordar que neste dataset so aparece as primeiras 14 picks

In [ ]:
dataFrame_auxiliar.createOrReplaceTempView("df_minutes")

# Then you can write all the SQL you want, as your json is seen as a regular table now.
sqlized_df = spark.sql(
    """
    SELECT *
    FROM df_minutes
    WHERE name =="Trae Young"
    """
)

sqlized_df.toPandas()

In [ ]:
#é necessario fazer join dos dois datasets através do reference
cond = [dataFrame3.reference == dataFrame_auxiliar.b_reference]
df=dataFrame3.join(dataFrame_auxiliar,cond, 'outer')
                          

In [ ]:
df.toPandas()

In [ ]:
df.createOrReplaceTempView("df")

# Then you can write all the SQL you want, as your json is seen as a regular table now.
sqlized_df = spark.sql(
    """
    SELECT *
    FROM df
    where b_reference is not null and reference is not null
    """
)

sqlized_df.toPandas() 
#existem 3558 linhas de players que foram as primeiras 14  escolhas e tiveram carreira no nba 

In [ ]:
df.createOrReplaceTempView("df")

# Then you can write all the SQL you want, as your json is seen as a regular table now.
sqlized_df = spark.sql(
    """
    SELECT *
    FROM df
    where b_reference is  null and reference is  null
    """
)

sqlized_df.toPandas()
#numero de jogadores que foram drafteados mas nunca jogaram  (nao ha nenhum lmao)

In [ ]:
df.createOrReplaceTempView("df")

# Then you can write all the SQL you want, as your json is seen as a regular table now.
sqlized_df = spark.sql(
    """
    SELECT *
    FROM df
    where b_reference is null and reference is not null
    """
)

sqlized_df.toPandas()
#numero de jogadores que foram as 14 primeiras escolhas do nba mas mesmo assim jogaram

In [ ]:
df.createOrReplaceTempView("df")

# Then you can write all the SQL you want, as your json is seen as a regular table now.
sqlized_df = spark.sql(
    """
    SELECT *
    FROM df
    where b_reference is not null and reference is  null
    """
)

sqlized_df.toPandas()
#numero de jogadores que foram as primeiras escolhas do draft mas nunca jogaram

In [ ]:
df.createOrReplaceTempView("df")

# Then you can write all the SQL you want, as your json is seen as a regular table now.
sqlized_df = spark.sql(
    """
    SELECT *
    FROM df
    where player == "Kostas Antetokounmpo"  or name =="Kostas Antetokounmpo"
    """
)

sqlized_df.toPandas()

In [ ]:
#TRATAR O TERCEIRO

In [ ]:
dataFrame_pl.toPandas()

In [ ]:
dataFrame_pl = dataFrame_pl.withColumnRenamed("_c0","columns") \
.withColumnRenamed("Player","player") 

In [ ]:
dataFrame_pl.createOrReplaceTempView("dataFrame_pl")

# Then you can write all the SQL you want, as your json is seen as a regular table now.
sqlized_df = spark.sql(
    """
    SELECT *
    FROM dataFrame_pl
    where Games =="NA" and PICK >=14 AND YEAR >=1995
    """
)

sqlized_df.show(217) 
sqlized_df.toPandas()
#Jogadores que nao realizaram nenhum jogo e foram drafteados depois do pick 14

In [ ]:
dataFrame_pl= dataFrame_pl.drop('columns')
dataFrame_pl= dataFrame_pl.drop('Play_Yrs')


In [ ]:
df_final = dataFrame_pl.filter((dataFrame_pl.Year >= "1995"))

df_final = df_final.filter((df_final.Games == "NA"))
df_final= df_final.filter((df_final.Pick >= "14"))

In [ ]:
#filtrar por aquilo que quero ano >= 1995, pick >= 14 e  games = NA
df_final.toPandas()

In [ ]:
df_final= df_final.withColumnRenamed("player","name_never_player") \
                  .withColumnRenamed("Games","games_NA")\
                  .withColumnRenamed("Year", "year_never_player")\
                  .withColumnRenamed("Pick","pick_never_played")

In [ ]:
#juntar as tabelas
cond = [df.reference == df_final.name_never_player]
df1=df.join(df_final,cond, 'outer')

In [ ]:
df1.toPandas()

In [ ]:
df1.printSchema() 

In [ ]:
#tratar o ultimo

In [ ]:
dataFrame_terciario.toPandas()

In [ ]:
import pyspark.sql.functions as f

df8 = dataFrame_terciario.withColumn("dia", f.split(dataFrame_terciario['Birthdate'], '/')[1]).\
           withColumn("mes", f.split(dataFrame_terciario['Birthdate'], '/')[0]).\
           withColumn("ano", f.split(dataFrame_terciario['Birthdate'], '/')[2])


df8.toPandas()

In [ ]:
#so preciso do basketball reference e do ano para fazer join
df9= df8.drop('Birthdate')
df9= df9.drop('dia')
df9= df9.drop('mes')


In [ ]:
df9.toPandas()

In [ ]:
df10= df9.withColumnRenamed("Basketball Reference ID","id") 

In [ ]:
#join
cond = [df.reference == df10.id]
df11=df1.join(df10,cond, 'outer')

In [ ]:
from pyspark.sql.functions import when, col, concat, lit

df11 = df11.withColumn(    
    "age_draft",
    col("draft_year")- col("ano")
)



In [ ]:
df11.toPandas()

In [ ]:
df11.select("age_draft").distinct().show()

In [ ]:
df11.createOrReplaceTempView("df11")

# Then you can write all the SQL you want, as your json is seen as a regular table now.
sqlized_df = spark.sql(
    """
    SELECT *
    FROM df11
    where age_draft is not null
    """
).toPandas()

In [ ]:
df11.printSchema()

In [ ]:
#necessario mudar os tipos 


from pyspark.sql.types import DoubleType
from pyspark.sql.types import *

df11 = df11.withColumn("Year", df11["Year"].cast(IntegerType())) 
df11 = df11.withColumn("age", df11["age"].cast(IntegerType()))
df11 = df11.withColumn("games", df11["games"].cast(IntegerType())) 
df11 = df11.withColumn("minutes_played", df11["minutes_played"].cast(DoubleType()))
df11 = df11.withColumn("player_efficiency", df11["player_efficiency"].cast(DoubleType())) 
df11 = df11.withColumn("true_shooting_percentage", df11["true_shooting_percentage"].cast(DoubleType())) 
df11 = df11.withColumn("proportion_of_shots_taken_3pts",df11["proportion_of_shots_taken_3pts"].cast(DoubleType())) 
df11 = df11.withColumn("free_throws_attempted_per_field_goal_attempt", df11["free_throws_attempted_per_field_goal_attempt"].cast(DoubleType())) 
df11 = df11.withColumn("offensive_rebounds_pct", df11["offensive_rebounds_pct"].cast(DoubleType())) 
df11 = df11.withColumn("defensive_rebounds_pct", df11["defensive_rebounds_pct"].cast(DoubleType())) 
df11 = df11.withColumn("trb_pct", df11["trb_pct"].cast(DoubleType())) 
df11 = df11.withColumn("assists_pct", df11["assists_pct"].cast(DoubleType())) 
df11 = df11.withColumn("steals_pct", df11["steals_pct"].cast(DoubleType())) 
df11 = df11.withColumn("blocks_pct", df11["blocks_pct"].cast(DoubleType())) 
df11 = df11.withColumn("Turnovers", df11["Turnovers"].cast(DoubleType())) 
df11 = df11.withColumn("usage_pct", df11["usage_pct"].cast(DoubleType()))
df11 = df11.withColumn("offensive_win_shares", df11["offensive_win_shares"].cast(DoubleType()))
df11 = df11.withColumn("defensive_win_shares", df11["defensive_win_shares"].cast(DoubleType())) 
df11 = df11.withColumn("bpm", df11["bpm"].cast(DoubleType()))
df11 = df11.withColumn("vorp", df11["vorp"].cast(DoubleType())) 
df11 = df11.withColumn("pick_overall", df11["pick_overall"].cast(IntegerType()))
df11 = df11.withColumn("draft_year", df11["draft_year"].cast(IntegerType()))
df11 = df11.withColumn("pick_never_played", df11["pick_never_played"].cast(IntegerType()))
df11 = df11.withColumn("ano", df11["ano"].cast(IntegerType())) 
df11 = df11.withColumn("win_shares", df11["win_shares"].cast(DoubleType()))
df11 = df11.withColumn("ws_per_48", df11["ws_per_48"].cast(DoubleType())) 
df11 = df11.withColumn("offensive_box_plus_minus", df11["offensive_box_plus_minus"].cast(DoubleType()))
df11 = df11.withColumn("defensive_box_plus_minus", df11["defensive_box_plus_minus"].cast(DoubleType())) 

In [ ]:
df11.toPandas()

In [ ]:
df11.printSchema()

In [ ]:
df11 \
    .select("*") \
    .write \
    .mode("overwrite") \
    .partitionBy("Year") \
    .format("delta") \
    .option("overwriteSchema", "true") \
    .save("hdfs://hdfs-nn:9000/projeto/NBA.db/sl_players_stats")